<a href="https://colab.research.google.com/github/Acottle1031/Sales-Predictions-Project/blob/main/SalesProjectionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
filename = '/content/drive/MyDrive/01 Week 1 - Python/Food Sales Predictions Project/sales_predictions.csv'

In [6]:
projections_df = pd.read_csv('/content/drive/MyDrive/01 Week 1 - Python/Food Sales Predictions Project/sales_predictions.csv')
projections_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [7]:
# 1) How many rows and columns?
#using df.shape we can see there are 8523 rows and 12 columns
projections_df.shape

(8523, 12)

In [9]:
# 2) What are the datatypes of each variable
# using df.dtypes we can see the individual data type for each variable
projections_df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [18]:
# 3) Are there duplicates? If so, drop any duplicates.
# By using df.duplicated().sum() we can have pandas go through all rows to check for duplicates and them show us the value of how many duplicates with the added .sum()
# Here we can see there are no duplicates
projections_df.duplicated().sum()


0

In [19]:
# 4) Identify missing values.
# by using .isna().sum() we can see the total number of missing values per column, so there are 1463 missing values in Item_Weight and 2410 missing in Outlet_Size
projections_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [29]:
# 5) Decide on how to address the missing values and do it! (This requires your judgement, so explain your choice).
# Outlet_Size is missing a lot of data, however this is most likley very useful to our sales projections so we want to keep the column
# We'll want to impute the average outlet size into the missing value spaces
#Item_weight also has a lot of missing values and may possibly be useful for projections so we'll also keep this
# Same thing as with Outlet_Size, we'll impute the average weight
outlet_size_average = projections_df['Outlet_Size'].mode().iloc[0]
item_weight_average = projections_df['Item_Weight'].mode
projections_df['Outlet_Size'].fillna(outlet_size_average, inplace=True)
projections_df['Item_Weight'].fillna(item_weight_average, inplace=True)

In [30]:
# 6) Confirm that there are no missing values after addressing them.
# like before, using .isna().sum() we can see the total number of all missing values in our dataframe after changes
projections_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [41]:
# 7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent) 
# using df.info we can see that all categories of data are unique and have no inconsistencies in them
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [45]:
# 8) For any numerical columns, obtain the summary statistics of each (min, max, mean)
# using df.describe we can see a summary of numerical columns including min, max, mean
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800
